In [1]:
import datetime

import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

import holidays

sns.set(font_scale=1.2)
%matplotlib inline

# Creating dataset

In this notebook I will create base dataset for training (without tuning for specific models).

In [2]:
items = pd.read_csv('../data/raw/items.csv')
item_categories = pd.read_csv('../data/raw/item_categories.csv')
shops = pd.read_csv('../data/raw/shops.csv')
sales_train = pd.read_csv('../data/raw/sales_train.csv')
test = pd.read_csv('../data/raw/test.csv')

## Cleaning

In this step I will clean dataset according to `1.0-db-EDA.ipynb`.

### Item categories

Now this section is empty, data was found clean.

### Shops

In [3]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


We found some typos in `shop_name`. We should replace shops with typos with shops without typos in `sales_train`, `test` (if it is the case).

In [4]:
map_typos = {0: 57, 1: 58, 10: 11, 39: 40}

Visualize mapping.

In [5]:
shops_list = shops.shop_name.to_list()
for from_id, to_id in map_typos.items():
    print(f"'{shops_list[from_id]}' ---> '{shops_list[to_id]}'")

'!Якутск Орджоникидзе, 56 фран' ---> 'Якутск Орджоникидзе, 56'
'!Якутск ТЦ "Центральный" фран' ---> 'Якутск ТЦ "Центральный"'
'Жуковский ул. Чкалова 39м?' ---> 'Жуковский ул. Чкалова 39м²'
'РостовНаДону ТРК "Мегацентр Горизонт"' ---> 'РостовНаДону ТРК "Мегацентр Горизонт" Островной'


Check datasets

In [6]:
for from_id, to_id in map_typos.items():
    print(f'Typos records with id={from_id} in train: {sales_train.shop_id.isin([from_id]).sum()}')
    print(f'Corrected records wit id={to_id} in train: {sales_train.shop_id.isin([to_id]).sum()}')

Typos records with id=0 in train: 9857
Corrected records wit id=57 in train: 117428
Typos records with id=1 in train: 5678
Corrected records wit id=58 in train: 71441
Typos records with id=10 in train: 21397
Corrected records wit id=11 in train: 499
Typos records with id=39 in train: 13440
Corrected records wit id=40 in train: 4257


In [7]:
for from_id, to_id in map_typos.items():
    print(f'Typos records with id={from_id} in test: {test.shop_id.isin([from_id]).sum()}')
    print(f'Corrected records wit id={to_id} in test: {test.shop_id.isin([to_id]).sum()}')

Typos records with id=0 in test: 0
Corrected records wit id=57 in test: 5100
Typos records with id=1 in test: 0
Corrected records wit id=58 in test: 5100
Typos records with id=10 in test: 5100
Corrected records wit id=11 in test: 0
Typos records with id=39 in test: 5100
Corrected records wit id=40 in test: 0


As we can see, in test there is just one possible variant from two. We have to select one of them. We select corrected variants in all cases.

**If we've made a mistake here, we will check it in the future (run without mapping)**

In [8]:
sales_train.shop_id[sales_train.shop_id.isin(map_typos.keys())] = sales_train.shop_id[
    sales_train.shop_id.isin(map_typos.keys())
].map(map_typos)

<ipython-input-8-913109016405>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_train.shop_id[sales_train.shop_id.isin(map_typos.keys())] = sales_train.shop_id[


In [9]:
test.shop_id[test.shop_id.isin(map_typos.keys())] = test.shop_id[
    test.shop_id.isin(map_typos.keys())
].map(map_typos)

Check online shops. Are they used in test dataset? If not we can just remove them to avoid dealing with Nans for some new features.

In [10]:
suspicious_shops = [9, 12, 55]
shops[shops.shop_id.isin(suspicious_shops)]

,shop_name,shop_id
9,Выездная Торговля,9
12,Интернет-магазин ЧС,12
55,Цифровой склад 1С-Онлайн,55


In [11]:
for shop in suspicious_shops:
    shop_name = shops[shops.shop_id == shop].shop_name.item()
    print(f"Records '{shop_name}' in train: {(sales_train.shop_id == shop).sum()}")

Records 'Выездная Торговля' in train: 3751
Records 'Интернет-магазин ЧС' in train: 34694
Records 'Цифровой склад 1С-Онлайн' in train: 34769


In [12]:
for shop in suspicious_shops:
    shop_name = shops[shops.shop_id == shop].shop_name.item()
    print(f"Records '{shop_name}' in train: {(test.shop_id == shop).sum()}")

Records 'Выездная Торговля' in train: 0
Records 'Интернет-магазин ЧС' in train: 5100
Records 'Цифровой склад 1С-Онлайн' in train: 5100


In this case we can remove `Выездная торговля`.

In [13]:
sales_train.shape

(2935849, 6)

In [14]:
sales_train = sales_train[sales_train.shop_id != 9]

### Items

In [15]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


We found typos with `!`, `*`, `/`, `D`.

#### Correct `!`

In [16]:
first_character = items.item_name.apply(lambda x: x[0])
typos_list = items[first_character == '!'].item_name.to_list()
typos_list

['! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D',
 '!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]']

There was no duplicates, thus just delete redundant `!`.

In [17]:
map_typos = {}
for typo in typos_list:
    map_typos[typo] = typo.strip('!').strip()
    
items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[
    items.item_name.isin(map_typos.keys())
].map(map_typos)

<ipython-input-17-3585e36035b3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[


#### Correct `*`

In [18]:
typos_list = items[first_character == '*'].item_name.to_list()
typos_list

['***В ЛУЧАХ СЛАВЫ   (UNV)                    D',
 '***ГОЛУБАЯ ВОЛНА  (Univ)                      D',
 '***КОРОБКА (СТЕКЛО)                       D',
 '***НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ  (UNI)             D',
 '***УДАР ПО ВОРОТАМ (UNI)               D',
 '***УДАР ПО ВОРОТАМ-2 (UNI)               D',
 '***ЧАЙ С МУССОЛИНИ                     D',
 '***ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI)             D',
 '*ЗА ГРАНЬЮ СМЕРТИ                       D',
 '*ЛИНИЯ СМЕРТИ                           D',
 '*МИХЕЙ И ДЖУМАНДЖИ  Сука любовь',
 '*СПАСАЯ ЭМИЛИ                           D',
 '*ЧОКНУТЫЙ ПРОФЕССОР /МАГИЯ/             D']

There was duplicate problem with `*МИХЕЙ И ДЖУМАНДЖИ  Сука любовь`. In all other cases we should just remove `*`.

In [19]:
map_typos = {}
for typo in typos_list:
    if typo[:3] != '*МИ':
        map_typos[typo] = typo.strip('*').strip()
    
items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[
    items.item_name.isin(map_typos.keys())
].map(map_typos)

<ipython-input-19-3fa1c6e4f692>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[


Case with duplicate we should process separately. Check how many records with both typo and corrected.

In [20]:
items[items.item_name.str.contains('МИХЕЙ')]

,item_name,item_id,item_category_id
12,*МИХЕЙ И ДЖУМАНДЖИ Сука любовь,12,55
14690,МИХЕЙ И ДЖУМАНДЖИ Сука любовь,14690,55
14691,МИХЕЙ И ДЖУМАНДЖИ Сука любовь LP,14691,58


In [21]:
map_typos = {12: 14690}

In [22]:
for from_id, to_id in map_typos.items():
    print(f'Typos records with id={from_id} in train: {sales_train.item_id.isin([from_id]).sum()}')
    print(f'Corrected records wit id={to_id} in train: {sales_train.item_id.isin([to_id]).sum()}')

Typos records with id=12 in train: 1
Corrected records wit id=14690 in train: 427


In [23]:
for from_id, to_id in map_typos.items():
    print(f'Typos records with id={from_id} in test: {test.item_id.isin([from_id]).sum()}')
    print(f'Corrected records wit id={to_id} in test: {test.item_id.isin([to_id]).sum()}')

Typos records with id=12 in test: 0
Corrected records wit id=14690 in test: 42


We can see, that we can make mapping without problems.

In [24]:
sales_train.item_id[sales_train.item_id.isin(map_typos.keys())] = sales_train.item_id[
    sales_train.item_id.isin(map_typos.keys())
].map(map_typos)

In [25]:
for from_id, to_id in map_typos.items():
    print(f'Typos records with id={from_id} in train: {sales_train.item_id.isin([from_id]).sum()}')
    print(f'Corrected records wit id={to_id} in train: {sales_train.item_id.isin([to_id]).sum()}')

Typos records with id=12 in train: 0
Corrected records wit id=14690 in train: 428


#### Correct `/`

In [26]:
typos_list = items[first_character == '/'].item_name.to_list()
typos_list

['//АДРЕНАЛИН: ОДИН ПРОТИВ ВСЕХ (Регион)',
 '//МОНГОЛ С.Бодров (Регион)',
 '//НЕ ОСТАВЛЯЮЩИЙ СЛЕДА (Регион)',
 '/БОМБА ДЛЯ НЕВЕСТЫ /2DVD/               D',
 '/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72',
 '/ОДНАЖДЫ В КИТАЕ-2',
 '/ПОСЛЕДНИЙ ШАНС',
 '/ПРОКЛЯТЬЕ ЭЛЬ ЧАРРО',
 '/СЕВЕР И ЮГ /Ч.2/',
 '/СМЕРТЕЛЬНЫЙ РАСКЛАД',
 '/ТЫ  - ТРУП',
 '/УМНОЖАЮЩИЙ ПЕЧАЛЬ т.2 (сер.3-4)']

There was no duplicates problem.

In [27]:
map_typos = {}
for typo in typos_list:
    map_typos[typo] = typo.strip('/')
    
items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[
    items.item_name.isin(map_typos.keys())
].map(map_typos)

<ipython-input-27-57fb3087080e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[


#### Correct `D`

In [28]:
typos_list = items[items.item_name.str.endswith('    D')].item_name.to_list()
typos_list

['ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D',
 'В ЛУЧАХ СЛАВЫ   (UNV)                    D',
 'ГОЛУБАЯ ВОЛНА  (Univ)                      D',
 'КОРОБКА (СТЕКЛО)                       D',
 'НОВЫЕ АМЕРИКАНСКИЕ ГРАФФИТИ  (UNI)             D',
 'УДАР ПО ВОРОТАМ (UNI)               D',
 'УДАР ПО ВОРОТАМ-2 (UNI)               D',
 'ЧАЙ С МУССОЛИНИ                     D',
 'ШУГАРЛЭНДСКИЙ ЭКСПРЕСС (UNI)             D',
 'ЗА ГРАНЬЮ СМЕРТИ                       D',
 'ЛИНИЯ СМЕРТИ                           D',
 'СПАСАЯ ЭМИЛИ                           D',
 'ЧОКНУТЫЙ ПРОФЕССОР /МАГИЯ/             D',
 'БОМБА ДЛЯ НЕВЕСТЫ /2DVD/               D']

There was no duplicates problem (it was already processed cases).

In [29]:
map_typos = {}
for typo in typos_list:
    map_typos[typo] = typo.strip('    D')
    
items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[
    items.item_name.isin(map_typos.keys())
].map(map_typos)

<ipython-input-29-e0db57958d99>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items.item_name[items.item_name.isin(map_typos.keys())] = items.item_name[


### Sales

In [30]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In EDA we have found entry with negativa price. 

In [31]:
sales_train[sales_train.item_price <= 1e-9]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
484683,15.05.2013,4,32,2973,-1.0,1.0


Look at another entries with this `shop_id` and `item_id`.

In [32]:
sales_train[(sales_train.shop_id == 32) & (sales_train.item_id == 2973)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
67427,29.01.2013,0,32,2973,2499.0,1.0
67428,25.01.2013,0,32,2973,2499.0,1.0
67429,22.01.2013,0,32,2973,2499.0,1.0
67430,21.01.2013,0,32,2973,2499.0,1.0
67431,18.01.2013,0,32,2973,2499.0,1.0
67432,17.01.2013,0,32,2973,2499.0,1.0
67433,15.01.2013,0,32,2973,2499.0,3.0
187844,05.02.2013,1,32,2973,2499.0,1.0
187845,14.02.2013,1,32,2973,2499.0,1.0
484682,23.05.2013,4,32,2973,1249.0,1.0


It obviously look like a mistake. Let's remove this row from the dataset.

In [33]:
sales_train = sales_train[sales_train.item_price > 0]

## Creating new features

In this step I will add new features according to `1.0-db-EDA.ipynb`.

### Item categories

In [34]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


#### Adding category and subcategory

We will split `item_category_name` by delimiter and treat parts as category and subcategory. If there is no delimiter, then it will be both: category and subcategory.

In [35]:
item_categories.item_category_name.to_list()[:10]

['PC - Гарнитуры/Наушники',
 'Аксессуары - PS2',
 'Аксессуары - PS3',
 'Аксессуары - PS4',
 'Аксессуары - PSP',
 'Аксессуары - PSVita',
 'Аксессуары - XBOX 360',
 'Аксессуары - XBOX ONE',
 'Билеты (Цифра)',
 'Доставка товара']

In [36]:
item_categories['category_name'] = item_categories.item_category_name.apply(lambda x: x.split(' - ')[0])
item_categories['subcategory_name'] = item_categories.item_category_name.apply(lambda x: x.split(' - ')[-1])

### Shops

In [37]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


#### Adding city

In [38]:
shops.shop_name.to_list()

['!Якутск Орджоникидзе, 56 фран',
 '!Якутск ТЦ "Центральный" фран',
 'Адыгея ТЦ "Мега"',
 'Балашиха ТРК "Октябрь-Киномир"',
 'Волжский ТЦ "Волга Молл"',
 'Вологда ТРЦ "Мармелад"',
 'Воронеж (Плехановская, 13)',
 'Воронеж ТРЦ "Максимир"',
 'Воронеж ТРЦ Сити-Парк "Град"',
 'Выездная Торговля',
 'Жуковский ул. Чкалова 39м?',
 'Жуковский ул. Чкалова 39м²',
 'Интернет-магазин ЧС',
 'Казань ТЦ "Бехетле"',
 'Казань ТЦ "ПаркХаус" II',
 'Калуга ТРЦ "XXI век"',
 'Коломна ТЦ "Рио"',
 'Красноярск ТЦ "Взлетка Плаза"',
 'Красноярск ТЦ "Июнь"',
 'Курск ТЦ "Пушкинский"',
 'Москва "Распродажа"',
 'Москва МТРЦ "Афи Молл"',
 'Москва Магазин С21',
 'Москва ТК "Буденовский" (пав.А2)',
 'Москва ТК "Буденовский" (пав.К7)',
 'Москва ТРК "Атриум"',
 'Москва ТЦ "Ареал" (Беляево)',
 'Москва ТЦ "МЕГА Белая Дача II"',
 'Москва ТЦ "МЕГА Теплый Стан" II',
 'Москва ТЦ "Новый век" (Новокосино)',
 'Москва ТЦ "Перловский"',
 'Москва ТЦ "Семеновский"',
 'Москва ТЦ "Серебряный Дом"',
 'Мытищи ТРК "XL-3"',
 'Н.Новгород ТРЦ

In [39]:
shops['city'] = shops.shop_name.apply(lambda x: x.strip('!').split()[0])

Delete city for `Выездная Торговля`.

Set city for `Интернет-магазин ЧС`, `Цифровой склад 1С-Онлайн` to `Online`.

In [40]:
shops.city[shops.shop_name == 'Выездная Торговля'] = None
shops.city[shops.shop_name.isin(['Интернет-магазин ЧС', 
                                 'Цифровой склад 1С-Онлайн'])] = 'Online'

<ipython-input-40-e7b125f1051c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shops.city[shops.shop_name == 'Выездная Торговля'] = None
<ipython-input-40-e7b125f1051c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shops.city[shops.shop_name.isin(['Интернет-магазин ЧС',


Add number of residents in a city.

In [41]:
num_residents = pd.read_csv('../data/external/num_residents.csv', index_col=0)
num_residents.head()

,num_residents
city,
Якутск,322987
Адыгея,463088
Балашиха,507366
Волжский,323906
Вологда,310302


In [42]:
shops['num_residents'] = shops.city.map(num_residents.num_residents)

#### Adding coordinates of a shop

To be added.

### Items

In [43]:
items.head()

,item_name,item_id,item_category_id
0,ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.),0,40
1,ABBYY FineReader 12 Professional Edition Full ...,1,76
2,В ЛУЧАХ СЛАВЫ (UNV),2,40
3,ГОЛУБАЯ ВОЛНА (Univ),3,40
4,КОРОБКА (СТЕКЛО),4,40


#### Addint text features

To be added in experiments.

### Sales

In [44]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


#### Adding date features

In [45]:
sales_train.date = pd.to_datetime(sales_train.date, format='%d.%m.%Y')

In [46]:
sales_train['day'] = sales_train.date.dt.day
sales_train['month'] = sales_train.date.dt.month
sales_train['year'] = sales_train.date.dt.year
sales_train['weekday'] = sales_train.date.dt.weekday

Add holiday features.

In [47]:
def days_since_holiday(datetimes):
    """Assign to each date days since last holiday."""
    ru_holidays = holidays.Russia()
    days_since_holiday = np.zeros(datetimes.shape[0])
    for i, current_datetime in tqdm(enumerate(datetimes.values), total=datetimes.size):
        last_holiday = current_datetime
        while not last_holiday in ru_holidays:
            last_holiday = last_holiday - datetime.timedelta(days=1)
    days_since_holiday[i] = (current_datetime - last_holiday).days
    return days_since_holiday

In [48]:
russian_holidays = holidays.Russia()
sales_train['is_holiday'] = sales_train.date.apply(lambda x: x in russian_holidays)

In [49]:
sales_train['days_since_holiday'] = days_since_holiday(
    sales_train.date.dt.date
).astype(int)

#### Adding price features

We will device price on "even" and "non even" parts. Even part is a price, derived from first digit, non-even -- residual.

In [50]:
sales_train['item_non_even_price'] = sales_train.item_price.apply(lambda x: float(str(x)[1:]))
sales_train['item_even_price'] = sales_train.item_price - sales_train['item_non_even_price']

In [51]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,weekday,is_holiday,days_since_holiday,item_non_even_price,item_even_price
0,2013-01-02,0,59,22154,999.00,1.0,2,1,2013,2,True,0,99.00,900.0
1,2013-01-03,0,25,2552,899.00,1.0,3,1,2013,3,True,0,99.00,800.0
2,2013-01-05,0,25,2552,899.00,-1.0,5,1,2013,5,True,0,99.00,800.0
3,2013-01-06,0,25,2554,1709.05,1.0,6,1,2013,6,True,0,709.05,1000.0
4,2013-01-15,0,25,2555,1099.00,1.0,15,1,2013,1,False,0,99.00,1000.0


## Aggregating

Aggregate all features in one table by months.